In [ ]:
import numpy as np
import pandas as pd
import threading
import datetime

def findFirstdayofMonth(yy,mm):
    firstday = datetime.date(yy,mm,1)
    return firstday
def findLastdayofMonth(yy,mm):
    if mm == 12:
        lastday = datetime.date(yy+1,1,1) - datetime.timedelta(days=1)
    else:
        lastday = datetime.date(yy,mm+1,1) - datetime.timedelta(days=1)
    return lastday

def process_excel_file(inputfile03,inputfile06,inputfile12,inputfile48,outputfile,info):

    start_time = info[0]
    end_time = info[1]
    month = start_time.strftime('%m')
    monthname = start_time.strftime('%#m') + '月'
    daysofmonth = (end_time - start_time).days + 1
    spcnamelist = ['不挥发','半挥发','低挥发']
    
    resultdict = {}
    resultList = []
    # 读取数据
    df03 = pd.read_excel(inputfile03)
    df12 = pd.read_excel(inputfile12)
    df06 = pd.read_excel(inputfile06)
    df48 = pd.read_excel(inputfile48)

    for jhour in range(0,daysofmonth*24):
        k03 = (jhour + 1)%3
        k12 = (jhour + 4)%12
        k06 = (jhour + 4)%6   
        k48 = (jhour + 40)%48
        tempValue_03T0,tempValue_12T0,tempValue_06T0,tempValue_48T0 = 0.,0.,0.,0.
        tempValue_03T1,tempValue_12T1,tempValue_06T1,tempValue_48T1 = 0.,0.,0.,0.
        tempValue_03T2,tempValue_12T2,tempValue_06T2,tempValue_48T2 = 0.,0.,0.,0.
        tempValue_03T3,tempValue_12T3,tempValue_06T3,tempValue_48T3 = 0.,0.,0.,0.
        tempValue_03T4,tempValue_12T4,tempValue_06T4,tempValue_48T4 = 0.,0.,0.,0.

        for spc in spcnamelist:
            tempValue_03T0 = tempValue_03T0 + df03.loc[jhour,spc+'_T0']
            tempValue_03T1 = tempValue_03T1 + df03.loc[jhour,spc+'_T1']
            tempValue_03T2 = tempValue_03T2 + df03.loc[jhour,spc+'_T2']
            tempValue_03T3 = tempValue_03T3 + df03.loc[jhour,spc+'_T3']
            tempValue_03T4 = tempValue_03T4 + df03.loc[jhour,spc+'_T4']
            tempValue_12T0 = tempValue_12T0 + df12.loc[jhour,spc+'_T0']
            tempValue_12T1 = tempValue_12T1 + df12.loc[jhour,spc+'_T1']
            tempValue_12T2 = tempValue_12T2 + df12.loc[jhour,spc+'_T2']
            tempValue_12T3 = tempValue_12T3 + df12.loc[jhour,spc+'_T3']
            tempValue_12T4 = tempValue_12T4 + df12.loc[jhour,spc+'_T4']
            tempValue_06T0 = tempValue_06T0 + df06.loc[jhour,spc+'_T0']
            tempValue_06T1 = tempValue_06T1 + df06.loc[jhour,spc+'_T1']
            tempValue_06T2 = tempValue_06T2 + df06.loc[jhour,spc+'_T2']
            tempValue_06T3 = tempValue_06T3 + df06.loc[jhour,spc+'_T3']
            tempValue_06T4 = tempValue_06T4 + df06.loc[jhour,spc+'_T4']
            tempValue_48T0 = tempValue_48T0 + df48.loc[jhour,spc+'_T0']
            tempValue_48T1 = tempValue_48T1 + df48.loc[jhour,spc+'_T1']
            tempValue_48T2 = tempValue_48T2 + df48.loc[jhour,spc+'_T2']
            tempValue_48T3 = tempValue_48T3 + df48.loc[jhour,spc+'_T3']
            tempValue_48T4 = tempValue_48T4 + df48.loc[jhour,spc+'_T4']
        totalValue = (tempValue_03T0 + tempValue_03T1 + tempValue_03T2 + tempValue_03T3 +
                    tempValue_12T2 + tempValue_12T3 + 
                    tempValue_06T2 + tempValue_06T3 + 
                    tempValue_48T1 + tempValue_48T2 + tempValue_48T3)
        Average_age = ((tempValue_03T0/totalValue)*(k03/2) + (tempValue_03T1/totalValue)*(k03+(3/2))+(tempValue_03T2/totalValue)*(k03+(9/2))+(tempValue_03T3/totalValue)*(k03+(15/2))+
                        (tempValue_12T2/totalValue)*(k12+(36/2))+(tempValue_12T3/totalValue)*(k12+(60/2))+
                        (tempValue_06T2/totalValue)*(k06+(18/2))+(tempValue_06T3/totalValue)*(k06+(30/2))+
                        (tempValue_48T1/totalValue)*(k48+(48/2))+(tempValue_48T2/totalValue)*(k48+(144/2))+(tempValue_48T3/totalValue)*(k48+(240/2)))
        resultList.append(Average_age)
    resultdict.update({info[2]:resultList})
    #输出excel结果
    table = pd.DataFrame(resultdict)
    writer = pd.ExcelWriter(outputfile)
    table.to_excel(writer,'Sheet1')
    writer.save()
        # df03 = pd.read_excel(r"E:\\article_fig\\age_dis\\New_version\\value\\forAge\\3\\" + month + citypoint+"_Draw.xlsx")
        # df12 = pd.read_excel(r"E:\\article_fig\\age_dis\\New_version\\value\\classified_value\\12\\"+month+"\\"+citypoint+"_"+monthname+"fenxi分类结果.xlsx")
        # df06 = pd.read_excel(r"E:\\article_fig\\age_dis\New_version\\value\\forAge\\6\\"+month+citypoint+"_Draw.xlsx")
        # df48 = pd.read_excel(r"E:\\article_fig\\age_dis\New_version\\value\\forAge\\48\\"+month+citypoint+"_Draw.xlsx")

folder = ['beijing','chengdu','guangzhou','haerbin','lanzhou','shanghai','wuhan']
inputfile03list = ["E:\\article_fig\\age_dis\\New_version\\value\\forAge\\3\\" + i + j+"_Draw.xlsx" for i in ['01','04','07','10'] for j in folder]
inputfile12list = ["E:\\article_fig\\age_dis\\New_version\\value\\classified_value\\12\\"+i+"\\"+j+"_"+ii+"fenxi分类结果.xlsx" for i,ii in zip(['01','04','07','10'],['1月','4月','7月','10月']) for j in folder]
inputfile06list = ["E:\\article_fig\\age_dis\\New_version\\value\\forAge\\6\\" + i + j+"_Draw.xlsx" for i in ['01','04','07','10'] for j in folder]
inputfile48list = ["E:\\article_fig\\age_dis\\New_version\\value\\forAge\\48\\" + i + j+"_Draw.xlsx" for i in ['01','04','07','10'] for j in folder]
outputfilelist = ["E:\\article_fig\\age_dis\\New_version\\value\\" + i + j+"_AGEResult.xlsx" for i in ['01','04','07','10'] for j in folder]

DateinfoList=[[findFirstdayofMonth(2021,i),findLastdayofMonth(2021,i),city] for i in [1,4,7,10] for city in folder]


threads = [threading.Thread(target=process_excel_file, args=(inputfile03list[i],inputfile06list[i],inputfile12list[i],inputfile48list[i],outputfilelist[i],DateinfoList[i])) for i in range(len(inputfile03list))]

for thread in threads:
    thread.start()
for thread in threads:
    thread.join()
print("all done")





all done
